# String equation example

## Analytic problem formulation

We consider a vibrating string on the segment $[0, 1]$, fixed on both sides, with input $u$ and output $\tilde{y}$ in the middle:
$$
\begin{align*}
    \partial_{tt} \xi(z, t)
    + d \partial_t \xi(z, t)
    - k \partial_{zz} \xi(z, t)
    & = \delta(z - \tfrac{1}{2}) u(t), & 0 < z < 1,\ t > 0, \\
    \partial_z \xi(0, t) & = 0, & t > 0, \\
    \partial_z \xi(1, t) & = 0, & t > 0, \\
    \tilde{y}(t) & = \xi(1/2, t), & t > 0.
\end{align*}
$$

## Semidiscretized formulation

Using the finite volume method on the equidistant mesh $0 = z_1 < z_2 < \ldots < z_{n + 1} = 1$, where $n = 2 n_2 - 1$, we obtain the semidiscretized formulation:
$$
\begin{align*}
    \ddot{x}_i(t)
    + d \dot{x}_i(t)
    - k \frac{x_{i - 1}(t) - 2 x_i(t) + x_{i + 1}(t)}{h^2}
    & = \frac{1}{h} \delta_{i, n_2} u(t), & i = 1, 2, 3, \ldots, n - 1, n, \\
    x_0(t) & = 0, \\
    x_{n + 1}(t) & = 0, \\
    y(t) & = x_{n_2}(t),
\end{align*}
$$
where $h = \frac{1}{n}$, $x_i(t) \approx \int_{z_i}^{z_{i + 1}} \xi(z, t) \, \mathrm{d}z$, and $y(t) \approx \tilde{y}(t)$.

Separating cases $i = 1$ and $i = n$ in the first equation, we find:
$$
\begin{alignat*}{6}
    \ddot{x}_1(t)
    + d \dot{x}_1(t)
    &
    && + 2 k n^2 x_1(t)
    && - k n^2 x_2(t)
    && = 0, \\
    \ddot{x}_i(t)
    + d \dot{x}_i(t)
    & - k n^2 x_{i - 1}(t)
    && + 2 k n^2 x_i(t)
    && - k n^2 x_{i + 1}(t)
    && = n \delta_{i, n_2} u(t),
    & i = 2, 3, \ldots, n - 1, \\
    \ddot{x}_n(t)
    + d \dot{x}_n(t)
    & - k n^2 x_{n - 1}(t)
    && + 2 k n^2  x_n(t)
    &&
    && = 0, \\
    &
    &&
    &&
    & y(t)
    & = x_{n_2}(t).
\end{alignat*}
$$

## Import modules

In [ ]:
from __future__ import absolute_import, division, print_function

import numpy as np
import scipy.linalg as spla
import scipy.sparse as sps
import scipy.integrate as spint
import matplotlib.pyplot as plt
%matplotlib inline

from pymor.discretizations.iosys import SecondOrderSystem
from pymor.reductors.bt import BTReductor
from pymor.reductors.lti import IRKAReductor
from pymor.reductors.sobt import (SOBTpReductor, SOBTvReductor, SOBTpvReductor, SOBTvpReductor,
                                  SOBTfvReductor, SOBTReductor)

import logging
logging.getLogger('pymor.algorithms.gram_schmidt.gram_schmidt').setLevel(logging.ERROR)

## Assemble $M$, $D$, $K$, $B$, $C_p$

In [ ]:
n2 = 50
n = 2 * n2 - 1  # dimension of the system

d = 10  # damping
k = 1   # stiffness

M = sps.eye(n, format='csc')

D = d * sps.eye(n, format='csc')

K = sps.diags([n * [2 * k * n ** 2],
               (n - 1) * [-k * n ** 2],
               (n - 1) * [-k * n ** 2]],
              [0, -1, 1],
              format='csc')

B = np.zeros((n, 1))
B[n2 - 1, 0] = n

Cp = np.zeros((1, n))
Cp[0, n2 - 1] = 1

## Second-order system

In [ ]:
so_sys = SecondOrderSystem.from_matrices(M, D, K, B, Cp)

print('n = {}'.format(so_sys.n))
print('m = {}'.format(so_sys.m))
print('p = {}'.format(so_sys.p))

In [ ]:
poles = so_sys.poles(force_dense=True)
fig, ax = plt.subplots()
ax.plot(poles.real, poles.imag, '.')
ax.set_title('System poles')
plt.show()

In [ ]:
np.max(poles.real)

In [ ]:
w = np.logspace(-1, 3, 1000)
fig, ax = SecondOrderSystem.mag_plot(so_sys, w=w)
ax.set_title('Bode plot of the full model')
plt.show()

In [ ]:
psv = so_sys.psv()
vsv = so_sys.vsv()
pvsv = so_sys.pvsv()
vpsv = so_sys.vpsv()
fig, ax = plt.subplots(2, 2, figsize=(12, 8), sharey=True)
ax[0, 0].semilogy(range(1, len(psv) + 1), psv, '.-')
ax[0, 0].set_title('Position singular values')
ax[0, 1].semilogy(range(1, len(vsv) + 1), vsv, '.-')
ax[0, 1].set_title('Velocity singular values')
ax[1, 0].semilogy(range(1, len(pvsv) + 1), pvsv, '.-')
ax[1, 0].set_title('Position-velocity singular values')
ax[1, 1].semilogy(range(1, len(vpsv) + 1), vpsv, '.-')
ax[1, 1].set_title('Velocity-position singular values')
plt.show()

In [ ]:
print('H_2-norm of the full model:   {}'.format(so_sys.norm()))
print('H_inf-norm of the full model: {}'.format(so_sys.norm('Hinf')))

## Position Second-Order Balanced Truncation (SOBTp)

In [ ]:
r = 10
sobtp_reductor = SOBTpReductor(so_sys)
rom_sobtp = sobtp_reductor.reduce(r, projection='bfsr')

In [ ]:
poles_rom_sobtp = rom_sobtp.poles(force_dense=True)
fig, ax = plt.subplots()
ax.plot(poles_rom_sobtp.real, poles_rom_sobtp.imag, '.')
ax.set_title("SOBTp reduced model's poles")
plt.show()

In [ ]:
err_sobtp = so_sys - rom_sobtp
print('H_2-error for the SOBTp ROM:     {}'.format(err_sobtp.norm()))
print('H_inf-error for the SOBTp ROM:   {}'.format(err_sobtp.norm('Hinf')))

In [ ]:
fig, ax = SecondOrderSystem.mag_plot((so_sys, rom_sobtp), w=w)
ax.set_title('Bode plot of the full and SOBTp reduced model')
plt.show()

In [ ]:
fig, ax = SecondOrderSystem.mag_plot(err_sobtp, w=w)
ax.set_title('Bode plot of the SOBTp error system')
plt.show()

## Velocity Second-Order Balanced Truncation (SOBTv)

In [ ]:
r = 10
sobtv_reductor = SOBTvReductor(so_sys)
rom_sobtv = sobtv_reductor.reduce(r, projection='bfsr')

In [ ]:
poles_rom_sobtv = rom_sobtv.poles(force_dense=True)
fig, ax = plt.subplots()
ax.plot(poles_rom_sobtv.real, poles_rom_sobtv.imag, '.')
ax.set_title("SOBTv reduced model's poles")
plt.show()

In [ ]:
err_sobtv = so_sys - rom_sobtv
print('H_2-error for the SOBTv ROM:     {}'.format(err_sobtv.norm()))
print('H_inf-error for the SOBTv ROM:   {}'.format(err_sobtv.norm('Hinf')))

In [ ]:
fig, ax = SecondOrderSystem.mag_plot((so_sys, rom_sobtv), w=w)
ax.set_title('Bode plot of the full and SOBTv reduced model')
plt.show()

In [ ]:
fig, ax = SecondOrderSystem.mag_plot(err_sobtv, w=w)
ax.set_title('Bode plot of the SOBTv error system')
plt.show()

## Position-Velocity Second-Order Balanced Truncation (SOBTpv)

In [ ]:
r = 10
sobtpv_reductor = SOBTpvReductor(so_sys)
rom_sobtpv = sobtpv_reductor.reduce(r, projection='bfsr')

In [ ]:
poles_rom_sobtpv = rom_sobtpv.poles(force_dense=True)
fig, ax = plt.subplots()
ax.plot(poles_rom_sobtpv.real, poles_rom_sobtpv.imag, '.')
ax.set_title("SOBTpv reduced model's poles")
plt.show()

In [ ]:
err_sobtpv = so_sys - rom_sobtpv
print('H_2-error for the SOBTpv ROM:     {}'.format(err_sobtpv.norm()))
#print('H_inf-error for the SOBTpv ROM:   {}'.format(err_sobtpv.norm('Hinf')))

In [ ]:
fig, ax = SecondOrderSystem.mag_plot((so_sys, rom_sobtpv), w=w)
ax.set_title('Bode plot of the full and SOBTpv reduced model')
plt.show()

In [ ]:
fig, ax = SecondOrderSystem.mag_plot(err_sobtpv, w=w)
ax.set_title('Bode plot of the SOBTpv error system')
plt.show()

## Velocity-Position Second-Order Balanced Truncation (SOBTvp)

In [ ]:
r = 10
sobtvp_reductor = SOBTvpReductor(so_sys)
rom_sobtvp = sobtvp_reductor.reduce(r, projection='bfsr')

In [ ]:
poles_rom_sobtvp = rom_sobtvp.poles(force_dense=True)
fig, ax = plt.subplots()
ax.plot(poles_rom_sobtvp.real, poles_rom_sobtvp.imag, '.')
ax.set_title("SOBTvp reduced model's poles")
plt.show()

In [ ]:
err_sobtvp = so_sys - rom_sobtvp
print('H_2-error for the SOBTvp ROM:     {}'.format(err_sobtvp.norm()))
print('H_inf-error for the SOBTvp ROM:   {}'.format(err_sobtvp.norm('Hinf')))

In [ ]:
fig, ax = SecondOrderSystem.mag_plot((so_sys, rom_sobtvp), w=w)
ax.set_title('Bode plot of the full and SOBTvp reduced model')
plt.show()

In [ ]:
fig, ax = SecondOrderSystem.mag_plot(err_sobtvp, w=w)
ax.set_title('Bode plot of the SOBTvp error system')
plt.show()

## Free-Velocity Second-Order Balanced Truncation (SOBTfv)

In [ ]:
r = 10
sobtfv_reductor = SOBTfvReductor(so_sys)
rom_sobtfv = sobtfv_reductor.reduce(r, projection='sr')

In [ ]:
poles_rom_sobtfv = rom_sobtfv.poles(force_dense=True)
fig, ax = plt.subplots()
ax.plot(poles_rom_sobtfv.real, poles_rom_sobtfv.imag, '.')
ax.set_title("SOBTfv reduced model's poles")
plt.show()

In [ ]:
err_sobtfv = so_sys - rom_sobtfv
print('H_2-error for the SOBTfv ROM:     {}'.format(err_sobtfv.norm()))
print('H_inf-error for the SOBTfv ROM:   {}'.format(err_sobtfv.norm('Hinf')))

In [ ]:
fig, ax = SecondOrderSystem.mag_plot((so_sys, rom_sobtfv), w=w)
ax.set_title('Bode plot of the full and SOBTfv reduced model')
plt.show()

In [ ]:
fig, ax = SecondOrderSystem.mag_plot(err_sobtfv, w=w)
ax.set_title('Bode plot of the SOBTfv error system')
plt.show()

## Second-Order Balanced Truncation (SOBT)

In [ ]:
r = 10
sobt_reductor = SOBTReductor(so_sys)
rom_sobt = sobt_reductor.reduce(r, projection='bfsr')

In [ ]:
poles_rom_sobt = rom_sobt.poles(force_dense=True)
fig, ax = plt.subplots()
ax.plot(poles_rom_sobt.real, poles_rom_sobt.imag, '.')
ax.set_title("SOBT reduced model's poles")
plt.show()

In [ ]:
err_sobt = so_sys - rom_sobt
print('H_2-error for the SOBT ROM:     {}'.format(err_sobt.norm()))
print('H_inf-error for the SOBT ROM:   {}'.format(err_sobt.norm('Hinf')))

In [ ]:
fig, ax = SecondOrderSystem.mag_plot((so_sys, rom_sobt), w=w)
ax.set_title('Bode plot of the full and SOBT reduced model')
plt.show()

In [ ]:
fig, ax = SecondOrderSystem.mag_plot(err_sobt, w=w)
ax.set_title('Bode plot of the SOBT error system')
plt.show()

## Balanced Truncation (BT)

In [ ]:
r = 10
bt_reductor = BTReductor(so_sys.to_lti())
rom_bt = bt_reductor.reduce(r, projection='bfsr')

In [ ]:
poles_rom_bt = rom_bt.poles(force_dense=True)
fig, ax = plt.subplots()
ax.plot(poles_rom_bt.real, poles_rom_bt.imag, '.')
ax.set_title("BT reduced model's poles")
plt.show()

In [ ]:
err_bt = so_sys.to_lti() - rom_bt
print('H_2-error for the BT ROM:     {}'.format(err_bt.norm()))
print('H_inf-error for the BT ROM:   {}'.format(err_bt.norm('Hinf')))

In [ ]:
fig, ax = SecondOrderSystem.mag_plot((so_sys, rom_bt), w=w)
ax.set_title('Bode plot of the full and BT reduced model')
plt.show()

In [ ]:
fig, ax = SecondOrderSystem.mag_plot(err_bt, w=w)
ax.set_title('Bode plot of the BT error system')
plt.show()